In [1]:
# from pyml4.common import context, db
# context.local_context.print_summary()
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

import gensim
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import pymysql.cursors
import json

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))
if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

In [2]:
def find_capitalization_dup(data, tag_col):
    new_data = data.copy()
    tag_lowercase_list = []
    tag_lowercase_dup_dict = {}

    # add a column that contains the lowercase version of tags
    for index, row in new_data.iterrows():
        tag = row[tag_col]
        tag_lower = tag.lower()
        tag_lowercase_list.append(tag_lower)
        if tag_lower not in tag_lowercase_dup_dict:
            tag_lowercase_dup_dict[tag_lower] = []
            tag_lowercase_dup_dict[tag_lower].append(tag)
        else:
            tag_lowercase_dup_dict[tag_lower].append(tag)

    new_data['tag_lowercase'] = tag_lowercase_list

    tag_lowercase_dup_list = []
    # identify capitalization duplicates
    print("identify capitalization duplicates")
    for index, row in new_data.iterrows():
        tag = row[tag_col]
        tag_lowercase = row[-1]
        temp = ""
        for item in tag_lowercase_dup_dict[tag_lowercase]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_lowercase_dup_list.append(temp)
        else:
            tag_lowercase_dup_list.append(None)
    # drop duplicates with lower scores
    new_data["tag_lowercase_dups"] = tag_lowercase_dup_list
    return new_data

In [83]:
def find_punctuation_dup(data, tag_col):
    new_data = data.copy()
    print("start processing tags with punctuations")
    import string

    table = str.maketrans('', '', string.punctuation)

    tag_strip_punc = []
    tag_punctuation_dup_dict = {}

    # Strip punctuation and store tag - tag_no_punctuation pairs
    for index, row in data.iterrows():
        tag = row[0]
        tag_lowercase = row[tag_col]
        stripped_tag = tag_lowercase.translate(table)
        tag_strip_punc.append(stripped_tag)
        if stripped_tag not in tag_punctuation_dup_dict:
            tag_punctuation_dup_dict[stripped_tag] = []
            tag_punctuation_dup_dict[stripped_tag].append(tag)
        else:
            tag_punctuation_dup_dict[stripped_tag].append(tag)
    new_data['tag_strip_punctuation'] = tag_strip_punc

    tag_punct_dup_list = []

    # print(data.head())
    
    for index, row in new_data.iterrows():
        tag = row[0]

        # make this a parameter
        tag_no_punc = row[-1]
        temp = ""
        for item in tag_punctuation_dup_dict[tag_no_punc]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_punct_dup_list.append(temp)
        else:
            tag_punct_dup_list.append(None)

    new_data["tag_punc_dups"] = tag_punct_dup_list
    return new_data

In [84]:
## TODO

def remove_whitespace_dup(data, tag_col):
    new_data = data.copy()
    print("start removing spaces")
    
    tag_no_spaces_list = []
    tag_no_spaces_dup_dict = {}

    # Strip punctuation and store tag - tag_no_punctuation pairs
    for index, row in new_data.iterrows():
        tag = row[0]
        tag_no_punc = row[tag_col]
        tag_no_space = tag_no_punc.replace(" ", "")

        tag_no_spaces_list.append(tag_no_space)
        if tag_no_space not in tag_no_spaces_dup_dict:
            tag_no_spaces_dup_dict[tag_no_space] = []
            tag_no_spaces_dup_dict[tag_no_space].append(tag)
        else:
            tag_no_spaces_dup_dict[tag_no_space].append(tag)
    new_data['tag_no_whitespace'] = tag_no_spaces_list

    tag_no_spaces_dup_list = []

    # print(data.head())
    
    for index, row in new_data.iterrows():
        tag = row[0]
        
        # make this a parameter
        tag_no_whitespace = row[-1]
        temp = ""
        for item in tag_no_spaces_dup_dict[tag_no_whitespace]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_no_spaces_dup_list.append(temp)
        else:
            tag_no_spaces_dup_list.append(None)

    new_data["tag_no_spaces_dups"] = tag_no_spaces_dup_list
    return new_data

In [19]:
def combine_columns(data, tag_col, col1, col2):
    dup_list = []
    for index, row in data.iterrows():
        tag = row[tag_col]
        capitalization_dup = row[col1]

        punctuation_dup = row[col2]

        #     print(capitalization_dup)
        #     print(punctuation_dup)
        if (capitalization_dup != None and punctuation_dup != None):
            dups = str(capitalization_dup)+str(punctuation_dup)
        elif capitalization_dup != None:
            dups = capitalization_dup
        elif punctuation_dup != None:
            dups = punctuation_dup
        else:
            dups = None
        dup_list.append(dups)

    new_data = data
    new_data['duplicates']=dup_list
    return new_data

In [7]:

def isEditDistanceTwo(s1, s2):

    # Find lengths of given strings
    m = len(s1)
    n = len(s2)

    # If difference between lengths is more than 1,
    # then strings can't be at one distance
    if abs(m - n) > 1:
        return False
    count = 0    # Count of isEditDistanceOne
    i = 0
    j = 0
    while i < m and j < n:
        # If current characters dont match
        if s1[i] != s2[j]:
            if count == 1:
                return False
            # If length of one string is
            # more, then only possible edit
            # is to remove a character
            if m > n:
                i+=1
            elif m < n:
                j+=1
            else:    # If lengths of both strings is same
                i+=1
                j+=1
            # Increment count of edits
            count+=1
        else:    # if current characters match
            i+=1
            j+=1
    # if last character is extra in any string
    if i < m or j < n:
        count+=1
    return count <= 2

In [35]:
def word2vec_and_2edits_away(data):
    print("start finding synonyms using word2vec")
    new_data = data.copy()
    tag_list = new_data['tag'].tolist()

    # load word2vedc model; edit the path and filename below;
    word_vec_model2 = Word2Vec.load('../models/Word2Vec_fullds_model2')

    # find synonyms using word2vec model;
    word2vec = []
    word2vec_2edits = []

    for index, row in new_data.iterrows():
        tag = row[0]
        temp1 = ''
        temp2 = ''
        if tag in word_vec_model2.wv.vocab:
            for (k,v) in word_vec_model2.wv.similar_by_word(tag):
                if k in tag_list:
                    #temp.append(k)
                    temp1 += k+";"
                    if isEditDistanceTwo(tag,k):
                        temp2 += k+";"
        if len(temp1) > 0:
            word2vec.append(temp1)
        else:
            word2vec.append(None)
        if len(temp2) > 0:
            word2vec_2edits.append(temp2)
        else:
            word2vec_2edits.append(None)
    # data = tag_no_cap_punc_stem.copy()
    new_data['word2vec'] = word2vec
    new_data['word2vec_2edits'] = word2vec_2edits
    return new_data

In [163]:
# Insert into mysql server:
def insert_to_sql(data_in_tuple, sql_table_name):
    ### Connect and insert to SQL

    import pymysql.cursors
    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")
    try:
        with connection.cursor() as cursor:
            sql_drop = "DELETE from %s" % (sql_table_name)
            cursor.execute(sql_drop)
            sql = "insert ignore into tag_replacement(tag,tag_lowercase_dups,tag_punctuation_dups,tag_no_spaces_dups) VALUES(%s, %s, %s, %s)"
            #number_of_rows = cursor.executemany(sql, data_to_insert)
            #% (tag_list,score_list,duplicates_list,word2vec_2edits_list,duplicates_with_word2vec_list)
            #for x in tuples:
            cursor.executemany(sql,data_in_tuple)
            #   cursor.execute(sql,x)

            connection.commit()
    finally:
        connection.close()

In [22]:
# load tag_movie from sql database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        sql = "SELECT tag,score FROM tag_movie"
        cursor.execute(sql)
        result = cursor.fetchall()

finally:
    connection.close()

movie_tag_app = pd.DataFrame(list(result), columns=['tag','score'])
movie_tag_app_agg = movie_tag_app.groupby("tag").sum().reset_index()


###### find capitalization dup
tag_capitalization_dups = find_capitalization_dup(movie_tag_app_agg, 0)
#tag_capitalization_dups.sort_values(by="score",ascending=False,inplace=True)
#tag_capitalization_dups.sort_values(by="tag_lowercase",inplace=True)

print("drop duplicates with lower scores")
#tag_no_capitalization_dup = tag_capitalization_dups.drop_duplicates("tag_lowercase",keep="first")
tag_no_capitalization_dup



identify capitalization duplicates
drop duplicates with lower scores


In [164]:
def main():
    # load tag_movie from sql database
    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")

    try:
        with connection.cursor() as cursor:
            sql = "SELECT tag,score FROM tag_movie"
            cursor.execute(sql)
            result = cursor.fetchall()

    finally:
        connection.close()

    movie_tag_app = pd.DataFrame(list(result), columns=['tag','score'])
    movie_tag_app_agg = movie_tag_app.groupby("tag").sum().reset_index()
    tag_no_cap_dup = find_capitalization_dup(movie_tag_app_agg, 0)

    
    tag_no_cap_punc_dup = find_punctuation_dup(tag_no_cap_dup, 2)
    tag_no_cap_punc_space_dup = remove_whitespace_dup(tag_no_cap_punc_dup, 4)

    # Note that find_punctuation_dup uses tag_lowercase;
    # remove_whitespace_dup uses tag_no_cap_punc_dups;
    tag_no_cap_punc_space_dup.sort_values(by=["tag_lowercase","score"],inplace=True)
    print("drop duplicates when tags are converted to lowercase")
    tag_no_cap_punc_space_dup.drop_duplicates("tag_lowercase",keep="last",inplace=True)

    #tag_no_cap_punc_space_dup2.sort_values(by="score",ascending=False,inplace=True)
    tag_no_cap_punc_space_dup.sort_values(by=["tag_strip_punctuation","score"],inplace=True)
    # print("drop duplicates when punctuations are removed")
    tag_no_cap_punc_space_dup.drop_duplicates("tag_strip_punctuation",keep="last",inplace=True)

    tag_no_cap_punc_space_dup.sort_values(by=["tag_no_whitespace","score"],inplace=True)
    tag_no_cap_punc_space_dup.drop_duplicates("tag_no_whitespace",keep="last",inplace=True)

    print(tag_no_cap_punc_space_dup.head())
    #### Insert into SQL server
    data_to_insert = tag_no_cap_punc_space_dup[["tag","tag_lowercase_dups","tag_punc_dups","tag_no_spaces_dups"]].copy()
    
    # convert np.nan to None type in python
    data_to_insert = data_to_insert.where(data_to_insert.notnull(), None)
    tuples = [tuple(x) for x in data_to_insert.values]
    insert_to_sql(tuples, "tag_replacement")
    

In [165]:
main()

identify capitalization duplicates
start processing tags with punctuations
start removing spaces
drop duplicates when tags are converted to lowercase
                         tag  score            tag_lowercase  \
756  =======================    2.0  =======================   
186                      007  163.0                      007   
190                 007-like    0.0                 007-like   
188      007 movies are bad.    0.0      007 movies are bad.   
189                007 rocks   -5.0                007 rocks   

    tag_lowercase_dups tag_strip_punctuation  \
756               None                         
186               None                   007   
190               None               007like   
188               None    007 movies are bad   
189               None             007 rocks   

                                         tag_punc_dups tag_no_whitespace  \
756  ************************;+++++++++++++++;.;:);...                     
186                     

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'tag_punctuation_dups' at row 1")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'tag_no_spaces_dups' at row 1")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1\\xBB\\x99 ph...' for column 'tag' at row 7084")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC4\\xB1lmaz' for column 'tag' at row 7693")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC8\\x99in\\xC4\\x83...' for column 'tag' at row 8384")
  result = self._query(query)
/Users/evazhong/anacond